<a href="https://colab.research.google.com/github/Shkum/IT_STEP/blob/ML/module3/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_%D0%BB%D1%96%D0%BD%D1%96%D0%B9%D0%BD%D0%B0_%D1%80%D0%B5%D0%B3%D1%80%D0%B5%D1%81%D1%96%D1%8F2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [133]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [134]:
df = pd.read_csv("https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/Used%20Car%20Dataset.csv", index_col="Unnamed: 0")
df.drop(columns=['car_name', 'registration_year', 'ownsership', 'transmission'], inplace=True)

Дані про ціни вживаних автомобілів у Індії
* car_name - назва авто у форматі 'рік компанія основна назва'
* registration_year - місяць та рік реєстрації
* ownsership - кількість власників у форматі 'Second owner'
* transmission - тип коробки передач
* insurance_validity - тип страховки
* fuel_type - вид палива
* seats - кількість сидінь
* kms_driven - пробіг
* manufacturing_year - рік виробництва, має тип object, потрібно застосувати astype(int)
* mileage(kmpl) - скільки кілометрів проїде за літр палива
* engine(cc) - об'єм двигуна у мл
* max_power(bhp) - потужність у кінських силах
* torque(Nm) - крутний момент двигуна
* price(in lakhs) - ціна у сто тисяч рупій

[Повний аналіз даних](https://www.kaggle.com/code/abdelrasoul/used-cars-prices-prediction)

In [135]:
df.head()

,insurance_validity,fuel_type,seats,kms_driven,manufacturing_year,mileage(kmpl),engine(cc),max_power(bhp),torque(Nm),price(in lakhs)
0,Comprehensive,Petrol,5,56000,2017,7.81,2996.0,2996.0,333.0,63.75
1,Comprehensive,Petrol,5,30615,2020,17.40,999.0,999.0,9863.0,8.99
2,Comprehensive,Diesel,5,24000,2018,20.68,1995.0,1995.0,188.0,23.75
3,Comprehensive,Petrol,5,18378,2019,16.50,1353.0,1353.0,13808.0,13.56
4,Comprehensive,Petrol,5,44900,2019,14.67,1798.0,1798.0,17746.0,24.00


In [136]:
df = df[['fuel_type', 'kms_driven', 'engine(cc)', 'price(in lakhs)']]
df.head()

,fuel_type,kms_driven,engine(cc),price(in lakhs)
0,Petrol,56000,2996.0,63.75
1,Petrol,30615,999.0,8.99
2,Diesel,24000,1995.0,23.75
3,Petrol,18378,1353.0,13.56
4,Petrol,44900,1798.0,24.00


# Завдання 1
Очистіть дані від викидів, розділіть на тренувальну та тестову чатини

In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1553 entries, 0 to 1552
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   fuel_type        1553 non-null   object 
 1   kms_driven       1553 non-null   int64  
 2   engine(cc)       1550 non-null   float64
 3   price(in lakhs)  1553 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 60.7+ KB


In [138]:
df.describe()

,kms_driven,engine(cc),price(in lakhs)
count,1553.000000,1.550000e+03,1553.000000
mean,52841.931101,1.471857e+10,166.141494
std,40067.800347,2.185629e+11,3478.855090
min,620.000000,5.000000e+00,1.000000
25%,30000.000000,1.197000e+03,4.660000
50%,49134.000000,1.462000e+03,7.140000
75%,70000.000000,1.995000e+03,17.000000
max,810000.000000,3.258640e+12,95000.000000


In [139]:
def remove_outliers(df, column_name):
  column = df[column_name]

  q1 = column.quantile(q=0.25)
  q3 = column.quantile(q=0.75)

  iqr = q3 - q1

  lower = q1 - 1.5*iqr
  upper = q3 + 1.5*iqr

  mask = (column >= lower) & (column <= upper)

  return df[mask]

for column_name in df.columns:
  if df.dtypes[column_name] in ("object", "datetime64[ns]"):
    continue

  df = remove_outliers(df, column_name)

In [140]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1223 entries, 1 to 1552
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   fuel_type        1223 non-null   object 
 1   kms_driven       1223 non-null   int64  
 2   engine(cc)       1223 non-null   float64
 3   price(in lakhs)  1223 non-null   float64
dtypes: float64(2), int64(1), object(1)
memory usage: 47.8+ KB


In [141]:
y = df['price(in lakhs)']
X = df.drop(columns='price(in lakhs)')

In [142]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.8,
                                                    random_state=42)

# Завдання 2
Створіть Pipeline для обробки числових стовпчиків(заповнення пропусків)

In [143]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

In [144]:
num_columns = X.select_dtypes(include="number").columns

In [145]:
num_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="median"))]
    )

# Завдання 3
Створіть Pipeline для обробки категоріальних стовпчиків(заповнення пропусків та кодування)

In [146]:
cat_columns = X.select_dtypes(include="object").columns

In [147]:
cat_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="most_frequent")),
           ("encoder", OneHotEncoder())
           ])

# Завдання 4
Об'єднайте попередні Pipeline в один ColumnTransformer

In [148]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_transformer, num_columns),
        ("cat", cat_transformer, cat_columns),
    ]
)

preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median'))]),
                                 Index(['kms_driven', 'engine(cc)'], dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder', OneHotEncoder())]),
                                 Index(['fuel_type'], dtype='object'))])

# Завдання 5
Створіть остаточну модель та натренуйте її

In [149]:
preprocessor.fit(X_train)
preprocessor.transform(X_train)

array([[6.1383e+04, 9.9800e+02, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [4.5000e+04, 1.1970e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [1.2000e+05, 1.9950e+03, 0.0000e+00, 0.0000e+00, 1.0000e+00,
        0.0000e+00],
       ...,
       [4.0909e+04, 1.1970e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [6.8888e+04, 1.1970e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [1.6937e+04, 1.1980e+03, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        1.0000e+00]])

In [150]:
model = Pipeline(
    [("prep", preprocessor),
     ("model", LinearRegression())
     ]
)

model

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['kms_driven', 'engine(cc)'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['fuel_type'], dtype='object'))])),
                ('model', LinearRegression())])

In [151]:
model.fit(X_train, y_train)

Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['kms_driven', 'engine(cc)'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder())]),
                                                  Index(['fuel_type'], dtype='object'))])),
                ('model', LinearRegression())])

# Завдання 6
Виведіть основні метрики моделі

In [152]:
y_pred = model.predict(X_test)

In [153]:
from sklearn import metrics

def get_metrics(y_true, y_pred, name="model"):
  df = pd.DataFrame()

  df.loc["MAE", name] = metrics.mean_absolute_error(y_true, y_pred)
  df.loc["MSE", name] = metrics.mean_squared_error(y_true, y_pred)
  # df.loc["RMSE", name] = metrics.mean_squared_error(y_true, y_pred) ** 0.5
  df.loc["R2", name] = metrics.r2_score(y_true, y_pred)

  return df.round(2)

In [154]:
all_metrics = pd.DataFrame()

In [155]:
all_metrics["train"] = get_metrics(y_train, model.predict(X_train))
all_metrics["test"] = get_metrics(y_test, y_pred)
all_metrics["mean"] = get_metrics(y_test, np.full_like(y_test, y_test.mean()), "mean")

In [156]:
all_metrics

,train,test,mean
MAE,3.86,3.67,4.92
MSE,30.00,29.05,44.23
R2,0.33,0.34,0.00


# Завдання 7
Збережіть модель

In [157]:
import joblib

In [158]:
joblib.dump(model, 'my_model')

['my_model']

In [159]:
my_model = joblib.load('my_model')

In [160]:
data = pd.DataFrame({'fuel_type': ['Diesel'],
                     'kms_driven': [48000],
                     'engine(cc)': [1832]})

data

,fuel_type,kms_driven,engine(cc)
0,Diesel,48000,1832


In [161]:
pred = my_model.predict(data)

In [162]:
pred

array([14.41288929])

In [167]:
sample = pd.DataFrame({'fuel_type': [np.nan], 'kms_driven': [np.nan], 'engine(cc)': [np.nan]})
sample

,fuel_type,kms_driven,engine(cc)
0,NaN,NaN,NaN


In [168]:
pred = my_model.predict(sample)

In [169]:
pred

array([7.17940508])